In [3]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.auth.transport.requests import Request
import os

SCOPES = ["https://www.googleapis.com/auth/drive.file"]

def autenticar():
    creds = None
    token_path = "keys/token.json"
    client_secret_path = "keys/google.json"

    # Cargar credenciales guardadas
    if os.path.exists(token_path):
        creds = Credentials.from_authorized_user_file(token_path, SCOPES)

    # Si no hay credenciales o ya no son válidas
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Este archivo debe ser el client_id descargado desde Google Console
            flow = InstalledAppFlow.from_client_secrets_file(
                client_secret_path, SCOPES)
            creds = flow.run_local_server(port=0)

        # Guardar el token para la próxima ejecución
        with open(token_path, 'w') as token:
            token.write(creds.to_json())
    return build('drive', 'v3', credentials=creds)


In [40]:
if __name__ == "__main__":
    service = autenticar()
    about = service.about().get(fields="user").execute()
    print("Autenticado como:", about['user']['displayName'])

Autenticado como: Santos Avila


In [53]:

def uploadToDrive(filepath_local, filename_drive, id_parent_folder, id_subfolder, mime_type="image/jpeg"):
    service = autenticar()

    if not id_parent_folder or not id_subfolder:
        raise ValueError("Debes proporcionar ambos: id_parent_folder y id_subfolder.")

    # Metadatos del archivo
    archivo_metadata = {
        'name': filename_drive,
        'parents': [id_subfolder]  # El archivo se sube al subfolder
    }

    media = MediaFileUpload(filepath_local, mimetype=mime_type)

    archivo = service.files().create(
        body=archivo_metadata,
        media_body=media,
        fields='id'
    ).execute()

    print(f"✅ Archivo subido con ID: {archivo.get('id')}")
    return archivo.get('id')



In [54]:
#  Ejemplo de uso
id_file = uploadToDrive(
    filepath_local="voucher_yape.jpeg",
    filename_drive="foto_1.jpeg",
    id_parent_folder="1Q0PBJiovZeRg8io6wCDr_lZnCf3_6ozb",
    id_subfolder="1yb6bhvkrofNl-cHBZPkeMoZex1QOC3VO" 
)

ruta = f"https://drive.google.com/drive/u/0/folders/{id_file}"
ruta

✅ Archivo subido con ID: 1BCKrCU6NEn8KD2PmRrueF400KP3v1Wvu


'https://drive.google.com/drive/u/0/folders/1BCKrCU6NEn8KD2PmRrueF400KP3v1Wvu'

In [ ]:


def searchByFolder(service, ruta, parent_id="root"):
    """
    Busca una carpeta en Google Drive desde 'Mi unidad', dada una ruta tipo 'Carpeta1/Subcarpeta2'.

    :param service: Objeto autenticado de Google Drive
    :param ruta: Ruta relativa desde "Mi unidad"
    :param parent_id: ID de la carpeta padre (por defecto 'root' = Mi unidad)
    :return: ID de la última carpeta encontrada o None
    """
    carpetas = ruta.strip("/").split("/")

    for nombre in carpetas:
        query = (
            f"mimeType='application/vnd.google-apps.folder' and "
            f"name='{nombre}' and "
            f"'{parent_id}' in parents and trashed=false"
        )
        resultado = service.files().list(q=query, fields="files(id, name)").execute()
        items = resultado.get("files", [])

        if not items:
            print(f"Carpeta no encontrada: {nombre}")
            return None

        parent_id = items[0]["id"]

    return parent_id

def listar_todas_las_carpetas(service):
    query = "mimeType='application/vnd.google-apps.folder' and trashed=false"
    
    resultado = service.files().list(
        q=query,
        spaces='drive',
        fields="files(id, name, parents)",
        pageSize=100
    ).execute()

    carpetas = resultado.get("files", [])
    
    if not carpetas:
        return [] 

    # Retornar una lista de diccionarios con nombre e ID
    return [{"name": c["name"], "id": c["id"]} for c in carpetas]

In [41]:
service = autenticar()
ruta = "Comprobantes-n8n"
carpeta_id = buscar_carpeta_por_ruta(service, ruta)

if carpeta_id:
    print("✅ ID de carpeta:", carpeta_id)
else:
    print("❌ Ruta no encontrada.")
service = autenticar()
ruta = "Facturas/Julio2025"
carpeta_id = buscar_carpeta_por_ruta(service, ruta)

if carpeta_id:
    print("✅ ID de carpeta:", carpeta_id)
else:
    print("❌ Ruta no encontrada.")


❌ Carpeta no encontrada: Comprobantes-n8n
❌ Ruta no encontrada.
❌ Carpeta no encontrada: Facturas
❌ Ruta no encontrada.


In [ ]:

listar_todas_las_carpetas(service)

[]

In [30]:
query = "mimeType='application/vnd.google-apps.folder' and trashed=false"
query


"mimeType='application/vnd.google-apps.folder' and trashed=false"

In [28]:
resultado = service.files().list(
    q=query,
    spaces='drive',
    fields="files(id, name, parents)",
    pageSize=100
).execute()

In [2]:
def obtener_id_root(service):
    about = service.about().get(fields="rootFolderId").execute()
    return about.get("rootFolderId")

service = autenticar()
root_id = obtener_id_root(service)
print(f"ID de la carpeta raíz: {root_id}")

NameError: name 'autenticar' is not defined

In [ ]:
def buscar_subcarpeta_por_nombre(nombre_carpeta, id_padre, creds):
    servicio = build('drive', 'v3', credentials=creds)

    query = f"""
        mimeType='application/vnd.google-apps.folder' and
        name='{nombre_carpeta}' and
        '{id_padre}' in parents and
        trashed=false
    """

    resultados = servicio.files().list(
        q=query,
        spaces='drive',
        fields="files(id, name)"
    ).execute()

    carpetas = resultados.get('files', [])
    return carpetas



AttributeError: 'Resource' object has no attribute 'authorize'

AttributeError: 'Resource' object has no attribute 'authorize'

In [23]:
import os
import json
import gspread
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

class GoogleService:
    def __init__(self):
        self.scopes = [
            "https://www.googleapis.com/auth/spreadsheets",
            "https://www.googleapis.com/auth/drive"
        ]
        
        ruta_json = os.path.join(os.getcwd(), "keys", "google.json")
        print(f"Leyendo credenciales desde: {ruta_json}")
        
        self.creds = ServiceAccountCredentials.from_json_keyfile_name(ruta_json, self.scopes)
        try:
            self.client = gspread.authorize(self.creds)
        except Exception as e:
            print(f"ERROR-AUTH-GOOGLE (gspread): {e}")
            self.client = None
        self.drive_service = build('drive', 'v3', credentials=self.creds)
        self.ParentFolder_id_comprobantes = "1XIJLgkoYUrDWJZkRwUac4DC-r4QugVre"

        self.MESES_ES = [
        "ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO",
        "JULIO", "AGOSTO", "SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"
    ]

    def pathByDate(self, fecha):
        """
        Retorna una ruta tipo 2025/JULIO/S2 según la fecha.
        """
        if isinstance(fecha, str):
            fecha = datetime.strptime(fecha, "%Y-%m-%d")

        year = fecha.year
        mes_nombre = self.MESES_ES[fecha.month - 1]

        # Semana relativa al inicio del mes
        primer_dia_mes = datetime(fecha.year, fecha.month, 1)
        dias_transcurridos = (fecha - primer_dia_mes).days
        semana = dias_transcurridos // 7 + 1

        return f"{year}/{mes_nombre}/S{semana}"    

    def buscar_carpeta(self, nombre_carpeta, id_padre):
        query = f"""
            mimeType='application/vnd.google-apps.folder' and
            name='{nombre_carpeta}' and
            '{id_padre}' in parents and
            trashed=false
        """
        resultados = self.drive_service.files().list(
            q=query,
            spaces='drive',
            fields="files(id, name)"
        ).execute()

        return resultados.get('files', [])

    def crear_carpeta(self, nombre_carpeta, id_padre):
        metadata = {
            'name': nombre_carpeta,
            'mimeType': 'application/vnd.google-apps.folder',
            'parents': [id_padre]
        }
        carpeta = self.drive_service.files().create(
            body=metadata,
            fields='id, name'
        ).execute()

        return carpeta

    def obtener_o_crear_carpeta(self, nombre_carpeta, id_padre):
        carpetas = self.buscar_carpeta(nombre_carpeta, id_padre)
        if carpetas:
            print(f"📁 Carpeta ya existe: {carpetas[0]['name']} (ID: {carpetas[0]['id']})")
            return carpetas[0]
        else:
            nueva = self.crear_carpeta(nombre_carpeta, id_padre)
            print(f"📁 Carpeta creada: {nueva['name']} (ID: {nueva['id']})")
            return nueva
        
    def listar_carpetas_en(self, id_padre):
        query = f"""
            mimeType='application/vnd.google-apps.folder' and
            '{id_padre}' in parents and
            trashed=false
        """
        resultado = self.drive_service.files().list(
            q=query,
            spaces='drive',
            fields='files(id, name)',
            pageSize=1000
        ).execute()

        carpetas = resultado.get('files', [])
        print(f"📂 Se encontraron {len(carpetas)} carpetas dentro de {id_padre}")
        return carpetas


In [ ]:
service.pathByDate()

In [14]:
if __name__ == "__main__":
    servicio = GoogleService()

    id_padre = "1XIJLgkoYUrDWJZkRwUac4DC-r4QugVre"  # carpeta "2025"
    
    # Obtener carpeta agosto o crearla si no existe
    #carpeta = servicio.obtener_o_crear_carpeta("AGOSTO", id_padre)

    # Listar carpetas dentro del padre
    carpetas = servicio.listar_carpetas_en(id_padre)
    for c in carpetas:
        print(f"{c['name']} (ID: {c['id']})")

🔐 Leyendo credenciales desde: c:\Users\USER\Documents\sancao-docs\crm-sancao\keys\google.json
📂 Se encontraron 1 carpetas dentro de 1XIJLgkoYUrDWJZkRwUac4DC-r4QugVre
2025 (ID: 1PbOlvizuN6JCahjR84CbueR_nHp2U0oq)


In [16]:


if __name__ == "__main__":
    servicio = GoogleService()

    id_padre = "1PbOlvizuN6JCahjR84CbueR_nHp2U0oq"  # carpeta "2025"
    
    # Obtener carpeta agosto o crearla si no existe
    #carpeta = servicio.obtener_o_crear_carpeta("AGOSTO", id_padre)

    # Listar carpetas dentro del padre
    carpetas = servicio.listar_carpetas_en(id_padre)
    for c in carpetas:
        print(f"{c['name']} (ID: {c['id']})")

🔐 Leyendo credenciales desde: c:\Users\USER\Documents\sancao-docs\crm-sancao\keys\google.json
📂 Se encontraron 3 carpetas dentro de 1PbOlvizuN6JCahjR84CbueR_nHp2U0oq
SEPTIEMBRE (ID: 1iIE-zq8LiDo6XiRaS4r3Q2pJ2NdNQCob)
AGOSTO (ID: 1FKGQNHZnzcM19xuKOQnUPpPQrR8oHjkq)
JULIO (ID: 1Q0PBJiovZeRg8io6wCDr_lZnCf3_6ozb)


In [ ]:
servicio.buscar_carpeta('JULIO', "1PbOlvizuN6JCahjR84CbueR_nHp2U0oq")


[{'id': '1Q0PBJiovZeRg8io6wCDr_lZnCf3_6ozb', 'name': 'JULIO'}]

In [ ]:
import os
import gspread
from datetime import datetime, timedelta, timezone
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

class GoogleService:
    def __init__(self):
        self.scopes = [
            "https://www.googleapis.com/auth/spreadsheets",
            "https://www.googleapis.com/auth/drive"
        ]
        ruta_json = os.path.join(os.getcwd(), "keys", "google.json")
        print(f"Leyendo credenciales desde: {ruta_json}")
        
        self.creds = ServiceAccountCredentials.from_json_keyfile_name(ruta_json, self.scopes)

        try:
            self.client = gspread.authorize(self.creds)
        except Exception as e:
            print(f"ERROR-AUTH-GOOGLE (gspread): {e}")
            self.client = None

        self.drive_service = build('drive', 'v3', credentials=self.creds)
        self.parentFolder_id_comprobantes = "1XIJLgkoYUrDWJZkRwUac4DC-r4QugVre"
        self.meses_es = [
        "ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO",
        "JULIO", "AGOSTO", "SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"
    ]

    def getPathFolderByDates(self, fecha):
        """
        Crea en Drive la ruta Comprobantes-n8n/AÑO/MES/Sx en zona horaria Perú.
        Devuelve el ID de la carpeta de la semana.
        """
        PERU_TZ = timezone(timedelta(hours=-5))
        if isinstance(fecha, str):
            fecha = datetime.strptime(fecha, "%Y-%m-%d")

        if fecha.tzinfo is None:
            fecha = fecha.replace(tzinfo=PERU_TZ)
        else:
            fecha = fecha.astimezone(PERU_TZ)

        year = str(fecha.year)
        month = self.meses_es[fecha.month - 1]
        week = f"S{((fecha.day - 1) // 7 + 1)}"

        path = f"{year}/{month}/{week}"
        return path
    
    def uploadToDriveByDate(self, path : str = "", file_info = None):
        year, month, week = path.split("/")
        carpeta_root = self.get_or_create_Folder("Comprobantes-n8n", "root")
        carpeta_year = self.get_or_create_Folder(year, carpeta_root["id"])
        carpeta_month = self.get_or_create_Folder(month, carpeta_year["id"])
        carpeta_week = self.get_or_create_Folder(week, carpeta_month["id"])

    def uploadToDriveByDate(self, fecha: str, file_info: dict):
        """
        Crea la ruta AÑO/MES/Sx en Drive basada en la fecha,
        sube el archivo y retorna dict con ID y URL.
        """
        path = self.getPathFolderByDates(fecha)
        year, month, week = path.split("/")

        # Crear carpetas dinámicas
        carpeta_root = self.get_or_create_Folder("Comprobantes-n8n", "root")
        carpeta_year = self.get_or_create_Folder(year, carpeta_root["id"])
        carpeta_month = self.get_or_create_Folder(month, carpeta_year["id"])
        carpeta_week = self.get_or_create_Folder(week, carpeta_month["id"])

        # Guardar archivo temporal
        filepath = f"out/temp_{file_info['nombre']}"
        with open(filepath, "wb") as f:
            f.write(base64.b64decode(file_info["base64"]))

        try:
            # Subir archivo
            file_id = uploadToDrive(
                filepath_local=filepath,
                filename_drive=file_info["nombre"],
                id_parent_folder=carpeta_week["id"],
                mime_type=self.obtener_mime_type(file_info["tipo"])
            )
        finally:
            os.remove(filepath)

        return {
            "id": file_id,
            "url": f"https://drive.google.com/file/d/{file_id}/view"
        }
    
    def get_or_create_Folder(self, nombre_carpeta, id_padre):
        carpetas = self.buscar_carpeta(nombre_carpeta, id_padre)
        if carpetas:
            print(f"Carpeta ya existe: {carpetas[0]['name']} (ID: {carpetas[0]['id']})")
            return carpetas[0]
        else:
            nueva = self.crear_carpeta(nombre_carpeta, id_padre)
            print(f"Carpeta creada: {nueva['name']} (ID: {nueva['id']})")
            return nueva

In [30]:
servicio = GoogleService()

id_final = servicio.getPathFolderByDates("2025-07-30")
        #print(f"Ruta lista. ID final: {id_final}")
id_final

Leyendo credenciales desde: c:\Users\USER\Documents\sancao-docs\crm-sancao\keys\google.json


('2025', 'JULIO', 'S5')